# Module 9. Scalable Data Processing

## Mandatory task 6. Large Airports.

### Pablo Yañez Martin

# RDD

In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
rutaA = "D:/Dropbox/Pablo/Master/Modulo 9. Scalable Data Processing/Java/Data/airports.csv"
rutaP = "D:/Dropbox/Pablo/Master/Modulo 9. Scalable Data Processing/Java/Data/countries.csv" 

In [3]:
spark_conf = SparkConf().setAppName("Airports_by_countryRDD").setMaster("local[2]")
spark_context = SparkContext(conf=spark_conf)

In [4]:
logger = spark_context._jvm.org.apache.log4j
logger.LogManager.getLogger("org").setLevel(logger.Level.WARN)

In [5]:
lecturaA = spark_context.textFile(rutaA)

In [6]:
lecturaP = spark_context.textFile(rutaP)

Now, I will prepare the rdds in order to join both rdd. The function join in rdd work as follow: the result of join (k1,v1) with (k2,v2) is (k1,(v1,v2)) where k1=k2.

In [7]:
columnasP=lecturaP.map(lambda line: line.split(','))

In [8]:
union1 = columnasP.map(lambda array : (array[1],array[2]))

In [9]:
columnasA=lecturaA.map(lambda line: line.split(','))

In [10]:
union2=columnasA.map(lambda array : (array[8],array[2]))

In [11]:
union=union1.join(union2)

In [12]:
resultado= union\
        .filter(lambda array : array[1][1]=="\"large_airport\"")\
        .map(lambda array: (array[1][0], 1))\
        .reduceByKey(lambda a, b: a + b)\
        .filter(lambda par : par[1]>10)\
        .sortBy(lambda pair: pair[1], ascending=False)

In [13]:
for (airport,num) in resultado.collect():
        print(airport,':',num)

"United States" : 170
"China" : 35
"United Kingdom" : 27
"Russia" : 21
"Italy" : 17
"Germany" : 16
"Turkey" : 13
"Japan" : 13
"Brazil" : 11
"India" : 11


In [14]:
spark_context.stop()

# DataFrame

In [15]:
from pyspark.sql import SparkSession, functions
from pyspark.sql.functions import explode, col

In [16]:
spark_session = SparkSession \
        .builder \
        .master("local[4]") \
        .getOrCreate()

In [17]:
logger = spark_session._jvm.org.apache.log4j
logger.LogManager.getLogger("org").setLevel(logger.Level.WARN)

In [18]:
dfA = spark_session \
        .read \
        .options(header='true', inferschema='true') \
        .option("delimiter", ",") \
        .csv(rutaA) \
        .persist()

In [19]:
dfP = spark_session \
        .read \
        .options(header='true', inferschema='true') \
        .option("delimiter", ",") \
        .csv(rutaP) \
        .persist()

As in both dataframe there is a column with the same name ('name') I will change that name.

In [20]:
dfA=dfA.withColumnRenamed("name","Airport name")

In [21]:
dfP=dfP.withColumnRenamed("name","Country name")

In [22]:
dfA.printSchema()

root
 |-- id: integer (nullable = true)
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- Airport name: string (nullable = true)
 |-- latitude_deg: double (nullable = true)
 |-- longitude_deg: double (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- scheduled_service: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- home_link: string (nullable = true)
 |-- wikipedia_link: string (nullable = true)
 |-- keywords: string (nullable = true)



In [23]:
dfA.show()

+------+-----+-------------+--------------------+------------------+-------------------+------------+---------+-----------+----------+------------+-----------------+--------+---------+----------+---------+--------------+--------+
|    id|ident|         type|        Airport name|      latitude_deg|      longitude_deg|elevation_ft|continent|iso_country|iso_region|municipality|scheduled_service|gps_code|iata_code|local_code|home_link|wikipedia_link|keywords|
+------+-----+-------------+--------------------+------------------+-------------------+------------+---------+-----------+----------+------------+-----------------+--------+---------+----------+---------+--------------+--------+
|  6523|  00A|     heliport|   Total Rf Heliport|    40.07080078125| -74.93360137939453|          11|       NA|         US|     US-PA|    Bensalem|               no|     00A|     null|       00A|     null|          null|    null|
|323361| 00AA|small_airport|Aero B Ranch Airport|         38.704022|        -101

In [24]:
dfA.schema.names

['id',
 'ident',
 'type',
 'Airport name',
 'latitude_deg',
 'longitude_deg',
 'elevation_ft',
 'continent',
 'iso_country',
 'iso_region',
 'municipality',
 'scheduled_service',
 'gps_code',
 'iata_code',
 'local_code',
 'home_link',
 'wikipedia_link',
 'keywords']

In [25]:
dfP.printSchema()

root
 |-- id: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- Country name: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- wikipedia_link: string (nullable = true)
 |-- keywords: string (nullable = true)



In [26]:
dfP.show()

+------+----+--------------------+---------+--------------------+--------------------+
|    id|code|        Country name|continent|      wikipedia_link|            keywords|
+------+----+--------------------+---------+--------------------+--------------------+
|302672|  AD|             Andorra|       EU|https://en.wikipe...|                null|
|302618|  AE|United Arab Emirates|       AS|https://en.wikipe...|UAE,مطارات في الإ...|
|302619|  AF|         Afghanistan|       AS|https://en.wikipe...|                null|
|302722|  AG| Antigua and Barbuda|       NA|https://en.wikipe...|                null|
|302723|  AI|            Anguilla|       NA|https://en.wikipe...|                null|
|302673|  AL|             Albania|       EU|https://en.wikipe...|                null|
|302620|  AM|             Armenia|       AS|https://en.wikipe...|                null|
|302556|  AO|              Angola|       AF|https://en.wikipe...|                null|
|302615|  AQ|          Antarctica|       AN

In [27]:
dfP.schema.names

['id', 'code', 'Country name', 'continent', 'wikipedia_link', 'keywords']

In [28]:
union = dfA.join(dfP,dfA["iso_country"]==dfP["code"])

In [29]:
union.printSchema()

root
 |-- id: integer (nullable = true)
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- Airport name: string (nullable = true)
 |-- latitude_deg: double (nullable = true)
 |-- longitude_deg: double (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- scheduled_service: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- home_link: string (nullable = true)
 |-- wikipedia_link: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- Country name: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- wikipedia_link: string (nullable = true)
 |-- keywords: string (nullable = true

Now i will drop all the columns that are not interesting.

In [30]:
union=union.drop("id")\
        .drop("ident")\
        .drop("latitude_deg")\
        .drop("longitude_deg")\
        .drop("elevation_ft")\
        .drop("continent")\
        .drop("municipality")\
        .drop("scheduled_service")\
        .drop("iata_code")\
        .drop("gps_code")\
        .drop("local_code")\
        .drop("home_link")\
        .drop("wikipedia_link")\
        .drop("keywords")\
        .drop("code")\
        .drop("iso_region")

In [31]:
union.printSchema()

root
 |-- type: string (nullable = true)
 |-- Airport name: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- Country name: string (nullable = true)



In [32]:
union.show()

+-------------+--------------------+-----------+-------------+
|         type|        Airport name|iso_country| Country name|
+-------------+--------------------+-----------+-------------+
|     heliport|   Total Rf Heliport|         US|United States|
|small_airport|Aero B Ranch Airport|         US|United States|
|small_airport|        Lowell Field|         US|United States|
|small_airport|        Epps Airpark|         US|United States|
|       closed|Newport Hospital ...|         US|United States|
|small_airport|      Fulton Airport|         US|United States|
|small_airport|      Cordes Airport|         US|United States|
|small_airport|Goldstone /Gts/ A...|         US|United States|
|small_airport| Williams Ag Airport|         US|United States|
|     heliport|Kitchen Creek Hel...|         US|United States|
|       closed|          Cass Field|         US|United States|
|small_airport| Grass Patch Airport|         US|United States|
|     heliport|  Ringhaver Heliport|         US|United 

In [33]:
union\
        .select("Country name","type")\
        .where(col("type")=="large_airport")\
        .groupBy("Country name")\
        .count()\
        .sort("count",ascending=False)\
        .where(col("count")>10)\
        .show()

+--------------+-----+
|  Country name|count|
+--------------+-----+
| United States|  170|
|         China|   35|
|United Kingdom|   27|
|        Russia|   21|
|         Italy|   17|
|       Germany|   16|
|        Turkey|   13|
|         Japan|   13|
|         India|   11|
|        Brazil|   11|
+--------------+-----+

